In [2]:
from math import sqrt

import numpy as np

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import skimage
from skimage import io, exposure, img_as_float, img_as_ubyte, morphology, filters, util
from skimage.color import rgb2gray, label2rgb
from skimage.feature import canny, blob_dog, blob_log, blob_doh,  peak_local_max
from skimage.filters import sobel, threshold_otsu, try_all_threshold, threshold_local, threshold_minimum
from skimage.segmentation import clear_border, watershed, random_walker
from skimage.measure import label, regionprops
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.morphology import closing

import cv2

from scipy import ndimage as ndi
from scipy.stats import norm

import os

import plotly.graph_objs as go

import pickle

In [6]:
with open("/Users/carolinemccue/Desktop/MG63/r01c02f26p03-ch2sk1fk1fl1/area5.pkl", 'rb') as f:
    area5 = pickle.load(f)

area5.head(5)

AttributeError: 'zip' object has no attribute 'head'

In [2]:
# 2.5 um

with open("/Users/carolinemccue/Desktop/MG63/r01c01f01p01-ch2sk1fk1fl1/labeled_carolinemccue.pkl", 'rb') as f:
    num_posts2p5 = pickle.load(f)
    
posts2p5 = {k:v for k,v in num_posts2p5.items() if v != 'Delete'}


In [5]:
# 5 um

with open("/Users/carolinemccue/Desktop/MG63/r01c02f26p03-ch2sk1fk1fl1/labeled_carolinemccue.pkl", 'rb') as f:
    num_posts5 = pickle.load(f)
    
posts5 = {k:v for k,v in num_posts5.items() if v != 'Delete'}

posts5

{'r01c02f26p03-ch2sk1fk1fl1_cell_67.jpg': '8',
 'r01c02f26p03-ch2sk1fk1fl1_cell_25.jpg': '11',
 'r01c02f26p03-ch2sk1fk1fl1_cell_86.jpg': '26',
 'r01c02f26p03-ch2sk1fk1fl1_cell_62.jpg': '8',
 'r01c02f26p03-ch2sk1fk1fl1_cell_21.jpg': '11',
 'r01c02f26p03-ch2sk1fk1fl1_cell_11.jpg': '13',
 'r01c02f26p03-ch2sk1fk1fl1_cell_60.jpg': '9',
 'r01c02f26p03-ch2sk1fk1fl1_cell_24.jpg': '6',
 'r01c02f26p03-ch2sk1fk1fl1_cell_0.jpg': '11',
 'r01c02f26p03-ch2sk1fk1fl1_cell_5.jpg': '12',
 'r01c02f26p03-ch2sk1fk1fl1_cell_57.jpg': '23',
 'r01c02f26p03-ch2sk1fk1fl1_cell_43.jpg': '14',
 'r01c02f26p03-ch2sk1fk1fl1_cell_66.jpg': '5',
 'r01c02f26p03-ch2sk1fk1fl1_cell_72.jpg': '8',
 'r01c02f26p03-ch2sk1fk1fl1_cell_23.jpg': '9',
 'r01c02f26p03-ch2sk1fk1fl1_cell_12.jpg': '9',
 'r01c02f26p03-ch2sk1fk1fl1_cell_1.jpg': '6',
 'r01c02f26p03-ch2sk1fk1fl1_cell_41.jpg': '12',
 'r01c02f26p03-ch2sk1fk1fl1_cell_9.jpg': '7',
 'r01c02f26p03-ch2sk1fk1fl1_cell_44.jpg': '9',
 'r01c02f26p03-ch2sk1fk1fl1_cell_22.jpg': '6',
 'r01c02

In [4]:
# 10 um

with open("/Users/carolinemccue/Desktop/MG63/r01c03f60p02-ch2sk1fk1fl1/labeled_carolinemccue.pkl", 'rb') as f:
    num_posts10 = pickle.load(f)
    
posts10 = {k:v for k,v in num_posts10.items() if v != 'Delete'}